## FastText

In [3]:
import os
import numpy as np
import pandas as pd
import fasttext

In [4]:
paths = {
    'MODELS':'models',
    'LR':os.path.join('models','LogisticRegression'),
    'FASTTEXT':os.path.join('models','fasttext'),
    'DATA':'data',
    'TRAIN':os.path.join('data','train'),
    'TEST':os.path.join('data','test')
}

### Загрузка предобработанных данных

In [5]:
df_gensim = pd.read_csv(os.path.join(paths['DATA'],'gensim.csv'), index_col=0)

In [6]:
df_cleaner = pd.read_csv(os.path.join(paths['DATA'],'cleaner.csv'), index_col=0)

In [7]:
df_gensim.head(5)

,text,topic
0,как стало известно агентству ассошиэйтед пресс...,Россия
1,зале игровых автоматов третьем ярусе подземног...,Россия
2,япония приняла решение разморозить кредиты рос...,Россия
3,британцы отмечают сегодня скорбную дату вторую...,Мир
4,понедельник директор департамента внешних связ...,Россия


In [8]:
df_cleaner.head(5)

,text,topic
0,как стало известно агентству ассошиэйтед пресс...,Россия
1,зале игровых автоматов третьем ярусе подземног...,Россия
2,япония приняла решение разморозить кредиты рос...,Россия
3,британцы отмечают сегодня скорбную дату вторую...,Мир
4,понедельник директор департамента внешних связ...,Россия


In [9]:
df_gensim['fasttext'] = df_gensim.apply(lambda x: f'__label__{x[1]} {x[0]}',axis=1)

In [10]:
df_cleaner['fasttext'] = df_cleaner.apply(lambda x: f'__label__{x[1]} {x[0]}',axis=1)

### Формирование тестовой и обучающей выборки с учетом несбалансированности классов

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [12]:
X_train_gensim, X_test_gensim, y_train_gensim, y_test_gensim = train_test_split(df_gensim.fasttext, 
                                                                                df_gensim.topic,  
                                                                                test_size=0.2, 
                                                                                stratify=df_gensim.topic,
                                                                                random_state=11)

In [13]:
pd.DataFrame(X_train_gensim).to_csv(os.path.join(paths['TRAIN'],'gensim_train.txt'), index=False)
pd.DataFrame(X_test_gensim).to_csv(os.path.join(paths['TEST'],'gensim_test.txt'), index=False)

In [14]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(df_cleaner.fasttext,
                                                                            df_cleaner.topic,
                                                                            test_size=0.2, 
                                                                            stratify=df_cleaner.topic, 
                                                                            random_state=11)

In [15]:
pd.DataFrame(X_train_clean).to_csv(os.path.join(paths['TRAIN'],'clean_train.txt'), index=False)
pd.DataFrame(X_test_clean).to_csv(os.path.join(paths['TEST'],'clean_test.txt'), index=False)

### Построение моделей

Модель для данных обработанных gensim

In [16]:
model_gensim = fasttext.train_supervised(os.path.join(paths['TRAIN'],'gensim_train.txt'))
model_gensim.test(os.path.join(paths['TEST'],'gensim_test.txt'))

(148220, 0.8460059371204965, 0.8460059371204965)

In [17]:
import re
predicts_gensim = []
for text in X_test_gensim:
    # model_gensim.predict(text.replace('__label__'))
    pridict = model_gensim.predict(re.sub('__label__[а-яА-Я_]*\s','',text))
    pridict = pridict[0][0].replace('__label__','')
    predicts_gensim.append(pridict)
print(classification_report(y_test_gensim, predicts_gensim))

                   precision    recall  f1-score   support

           Бизнес       0.68      0.43      0.53      1480
      Бывший_СССР       0.83      0.83      0.83      7094
              Дом       0.87      0.85      0.86      4347
         Из_жизни       0.70      0.67      0.69      5521
   Интернет_и_СМИ       0.80      0.76      0.78      8933
         Культура       0.90      0.90      0.90     10759
              Мир       0.83      0.86      0.84     27324
  Наука_и_техника       0.86      0.87      0.87     10627
      Путешествия       0.80      0.72      0.76      1281
           Россия       0.83      0.84      0.84     32088
Силовые_структуры       0.72      0.63      0.67      3919
            Спорт       0.97      0.97      0.97     12883
          Украина       0.87      0.87      0.87      4505
         Ценности       0.91      0.87      0.89      1553
        Экономика       0.86      0.88      0.87     15906

         accuracy                           0.85    14

In [18]:
model_gensim.save_model(os.path.join(paths['FASTTEXT'],'gensim.bin'))

Модель для данных обработанных классом CleanText

In [19]:
model_clean = fasttext.train_supervised(os.path.join(paths['TRAIN'],'clean_train.txt'))
model_clean.test(os.path.join(paths['TEST'],'clean_test.txt'))

(148220, 0.8462690595061395, 0.8462690595061395)

In [24]:
predicts_clean = []
for text in X_test_clean:
    # model_gensim.predict(text.replace('__label__'))
    pridict = model_clean.predict(re.sub('__label__[а-яА-Я_]*\s','',text))
    pridict = pridict[0][0].replace('__label__','')
    predicts_clean.append(pridict)
print(classification_report(y_test_clean, predicts_clean))

                   precision    recall  f1-score   support

           Бизнес       0.69      0.43      0.53      1480
      Бывший_СССР       0.83      0.82      0.83      7094
              Дом       0.88      0.86      0.87      4347
         Из_жизни       0.71      0.67      0.69      5521
   Интернет_и_СМИ       0.81      0.76      0.79      8933
         Культура       0.90      0.91      0.90     10759
              Мир       0.83      0.86      0.84     27324
  Наука_и_техника       0.86      0.87      0.87     10627
      Путешествия       0.81      0.71      0.76      1281
           Россия       0.83      0.85      0.84     32088
Силовые_структуры       0.72      0.63      0.67      3919
            Спорт       0.97      0.97      0.97     12883
          Украина       0.87      0.87      0.87      4505
         Ценности       0.91      0.86      0.89      1553
        Экономика       0.86      0.88      0.87     15906

         accuracy                           0.85    14

In [21]:
model_clean.save_model(os.path.join(paths['FASTTEXT'],'cleaner.bin'))

В целом результаты получились идентичные, однако предобработка написанным классом была быстрее.

In [22]:
model_clean1 = fasttext.load_model(os.path.join(paths['FASTTEXT'],'cleaner.bin'))